```
Notes:
Need to do the whole shebang with all the images
use color and resize with the ImageDataGenerator from keras on the fly
Will give us that extra dimension we need. 
```

## Review

Our last classsifier was very poor--it operated at chance--a coin flip would have had the same predictive power. A few things may have been going on that casued us to find no signal. I could be we down sampled our images too much and lost useful information, it could be that our model was poorly configured (it was), it could be we were using thr wrong model (we were), or it could be all of these. My gut (and eyes) say there is still plenty of information in our down sampled images, so we're going to try a more approproate model before we go back and try with larger images. 

We're going to build a convolutional neural network using keras (with Tensorflow as the backend). For starters, convolutional neural networks are much better at image classification than the mutli-layer perceptron used in the last post. 

In [2]:
# keeps warnings from printing for nicer blog output
import warnings 
#warnings.filterwarnings('ignore') 

import os

import pickle # for loading our extracted features
import pandas as pd # for DataFame class our features are loaded into to
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

# for processing images
from skimage.transform import rescale 

In [10]:
os.chdir(os.path.expanduser('~/share/rkingdc-blog/regplot'))

,type,id,n,n_predictors,n_controls,filename,d_biased,d_ceiling,d_none,d_outlier,img_series
0,none,1,5426,3,2,data/png/none_0000001.png,0,0,1,0,"[[57686, 61485, 61486, 61486, 61486, 61486, 61..."
1,none,2,588,3,4,data/png/none_0000002.png,0,0,1,0,"[[57686, 61485, 61486, 61486, 61486, 61486, 61..."
2,none,3,7519,4,3,data/png/none_0000003.png,0,0,1,0,"[[57686, 61485, 61486, 61486, 61486, 61486, 61..."
3,none,4,3831,4,3,data/png/none_0000004.png,0,0,1,0,"[[57686, 61485, 61486, 61486, 61486, 61486, 61..."
4,none,5,6888,4,3,data/png/none_0000005.png,0,0,1,0,"[[57686, 61485, 61486, 61486, 61486, 61486, 61..."


We can see this is the same file we saw in the previous post, but with the image data as an added column. Each cell in that `img_series` column is a numpy ndarray of shape `(168,168)`. 

## Building the Model

We'll start by building our classifier. Code will be explained inline. 

In [ ]:
## mod = Sequential()

# create our first layer with some sensible defaults. 
# we can tweak these later if needed
mod.add(Conv2D(filters=64, 
               kernel_size=(2,2), 
               input_shape=(126,126,3), # we've got 168x168 px photos in color
               activation='relu'))

# pooling layer reduces model complexity--good to run before flattening 
mod.add(MaxPooling2D(pool_size = (2,2)))

mod.add(Flatten())

# our hidden layer
mod.add(Dense(units=64, activation='relu'))

# our output layer
# binary classification (for now) so we just need one layer 
# sigmoid forces 0,1 output
mod.add(Dense(units=1, activation='sigmoid'))
mod.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
mod.summary()

## Pre-procesing Data

In image processing we'll want to pre-proccess our images before we train a model on them, by adding some random stretching, blurring, rotating, etc. Keras has utilities included to make this easier. 

In [7]:
def preproc(img):
    return rescale(img, .35, anti_aliasing=True)

train_gen = ImageDataGenerator(zoom_range=.2,
                              horizontal_flip=True,
                              width_shift_range=.2,
                              height_shift_range=.2,
                           #   preprocessing_function=preproc,
                              rescale=1/255)
test_gen = ImageDataGenerator(rescale=1/255,
                           #   preprocessing_function=preproc
                              )

In [ ]:
train_gen.flow_from_directory('data/imgs',
                             target_size=(126,126))
test_gen.flow_from_directory('data/imgs',
                             target_size=(126,126))

In [141]:
mod.fit(x=x_train,
        y=y_train,
        epochs=10,
        verbose=0,
        validation_data=(x_test, y_test))

ValueError: Error when checking input: expected conv2d_14_input to have 4 dimensions, but got array with shape (45000, 168, 168)

In [104]:
mod.layers

In [ ]:
mod.summa